In [79]:
#import necessary libraries
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

In [69]:
#import stock data from Quandl API
##change this to Alpaca if we do minute-by-minute
import quandl
quandl.ApiConfig.api_key = "tC5AjCkyqtgbMarYtnVU"

In [70]:
#set dates between which stock data will be pulled for a given ticker
START_PULL = "2005-01-01"
END_PULL = "2018-02-01"

In [71]:
#Adobe
adobeSymbol = "WIKI/ADBE"
adobeDf = quandl.get(adobeSymbol, start_date= START_PULL, end_date= END_PULL)

In [72]:
#cut-off for where data transitions from training set to test set
percent_train = 0.85
TRAIN_SPLIT = int(percent_train*len(adobeDf))
print(TRAIN_SPLIT)

full_data = len(adobeDf)
print(full_data)

features_considered = ['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']

features = adobeDf[features_considered]
features

2798
3292


Adj. Open   Adj. High    Adj. Low  Adj. Close  Adj. Volume
Date                                                                   
2005-01-03   31.494084   31.744037   30.769220   30.839207    5508800.0
2005-01-04   31.099158   31.184142   29.674425   30.024360    7515400.0
2005-01-05   30.044356   30.444281   29.859391   29.859391    3566600.0
2005-01-06   30.109344   30.164333   29.239507   29.364484    6159600.0
2005-01-07   29.634433   29.749411   28.844581   29.384480    8512400.0
...                ...         ...         ...         ...          ...
2018-01-26  198.500000  201.550000  198.250000  201.300000    2262478.0
2018-01-29  200.110000  200.860000  197.870000  198.230000    1876216.0
2018-01-30  197.250000  197.730000  194.890000  196.900000    3034232.0
2018-01-31  197.130000  200.960000  196.750000  199.760000    2514574.0
2018-02-01  199.120000  201.750000  198.084500  199.380000    2353675.0

[3292 rows x 5 columns]

In [73]:
#scale features before training NN by subtractic mean and dividing by stdev of each feature
#or use tf.keras.utils.normalize to rescale values to [0,1]
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis = 0)
data_std = dataset[:TRAIN_SPLIT].std(axis = 0)

dataset = (dataset-data_mean)/data_std
dataset

array([[-0.62426727, -0.63460294, -0.64110267, -0.66260519, -0.05483864],
       [-0.64686887, -0.66642972, -0.70422182, -0.70923479,  0.39884239],
       [-0.70723518, -0.70848653, -0.69355786, -0.71867514, -0.49395919],
       ...,
       [ 8.86194321,  8.8007382 ,  8.82109939,  8.84022113, -0.61432462],
       [ 8.85507561,  8.984345  ,  8.92833564,  9.00388459, -0.73181638],
       [ 8.9689633 ,  9.02925193,  9.00527476,  8.9821391 , -0.76819475]])

In [74]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                     target_size, step, single_step = False):
    data = []
    labels = []
    
    start_index = start_index+history_size
    if end_index is None:
        end_index = len(dataset) - target_size
        
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        
        if single_step:
            labels.append(dataset[indices])
        else:
            labels.append(target[i:i+target_size])
            
    return np.array(data), np.array(labels)
            

In [75]:
#historic data -- not entirely sure how this works since we wanna do over all of train_split
past_history = 1
#how far into the future we want to predict a price
future_target = 1
#how we group the data -- could be 5 if we group by week
#^^in which case future target woule be 1*5
STEP = 1


In [76]:
x_train_single, y_train_single = multivariate_data(dataset, dataset[:,1], 0, 
                                                   TRAIN_SPLIT, past_history, future_target, STEP,
                                                  single_step = True)

x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [77]:
print('single window of past histroy = {}'.format(x_train_single[0].shape))


single window of past histroy = (1, 5)


In [88]:
BATCH_SIZE = 5
BUFFER_SIZE = len(adobeDf)

train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [89]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

In [90]:
#how often the epoch will run -- so like 
#we can do over a fraction of the data set instead of the whole thing each time
#arbitraray-ish
EVALUATION_INTERVAL = int(TRAIN_SPLIT/5)
EPOCHS = 10

single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50)

Epoch 1/10
559/559 [==============================] - 2s 4ms/step - loss: 0.6722 - val_loss: 3.0543
Epoch 2/10
559/559 [==============================] - 2s 3ms/step - loss: 0.6667 - val_loss: 2.9694
Epoch 3/10
559/559 [==============================] - 2s 4ms/step - loss: 0.6675 - val_loss: 3.0369
Epoch 4/10
559/559 [==============================] - 2s 4ms/step - loss: 0.6659 - val_loss: 3.0228
Epoch 5/10
559/559 [==============================] - 3s 5ms/step - loss: 0.6668 - val_loss: 2.9696
Epoch 6/10
559/559 [==============================] - 2s 3ms/step - loss: 0.6642 - val_loss: 2.9371
Epoch 7/10
559/559 [==============================] - 2s 4ms/step - loss: 0.6638 - val_loss: 3.0309
Epoch 8/10
559/559 [==============================] - 2s 4ms/step - loss: 0.6647 - val_loss: 2.8786
Epoch 9/10
559/559 [==============================] - 2s 3ms/step - loss: 0.6672 - val_loss: 2.8870
Epoch 10/10
559/559 [==============================] - 2s 4ms/step - loss: 0.6677 - val_loss: 2.9846